
# Preprocessing - Integration
Michael Sterr

2022-09-21 13:29:49 


# Setup

Run following scripts before:
 * 05-2_scRNA-seq_iPSC_IIR-KO_S6_Preprocessing_Normalization_WT-IIR-KO_v5

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

In [2]:
# General
import scipy as sci
import numpy as np
import pandas as pd
import logging
import time
import pickle
from itertools import chain
import session_info
import gc # Free memory #gc.collect()
import scipy.stats as stats

# Plotting
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import rcParams
from matplotlib import cm
from matplotlib.pyplot import rc_context
import seaborn as sb
from adjustText import adjust_text

# Analysis
import anndata as ad
import scanpy as sc
import scvi
import scanpy.external as sce

Global seed set to 0


In [3]:
# Settings

## Scanpy settings
sc.settings.verbosity = 3
sc.logging.print_versions()
session_info.show()

-----
anndata     0.8.0
scanpy      1.9.1
-----
PIL                         8.4.0
absl                        NA
adjustText                  NA
anyio                       NA
astunparse                  1.6.3
attr                        21.2.0
babel                       2.9.1
backcall                    0.2.0
beta_ufunc                  NA
binom_ufunc                 NA
bottleneck                  1.3.2
certifi                     2022.06.15
cffi                        1.15.0
chardet                     4.0.0
charset_normalizer          2.0.7
chex                        0.1.1
cloudpickle                 2.0.0
colorama                    0.4.4
cupy                        10.1.0
cupy_backends               NA
cupyx                       NA
cycler                      0.10.0
cython_runtime              NA
dask                        2021.10.0
dateutil                    2.8.2
debugpy                     1.4.1
decorator                   5.1.0
defusedxml                  0.7.1
deprecate  

In [4]:
# Color maps
exec(open("/home/michi/Software/viscm/maps/michi_bk_bl_gn_yl.py").read())

In [5]:
# Plot settings
%matplotlib inline

## Directory
sc.settings.figdir='/home/michi/Projects/scRNA-seq_iPSC_IGFRL-KO_Notebooks/Figures'

## Plotting parameters
rcParams['figure.figsize']=(20,20) #rescale figures
#sc.set_figure_params(scanpy=True, frameon=False, vector_friendly=False, color_map='tab10' ,transparent=True, dpi=150, dpi_save=300)
sc.set_figure_params(scanpy=True, frameon=False, vector_friendly=False ,transparent=True, dpi=150, dpi_save=300)

## Font
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Source Sans 3']

## Grid & Ticks
rcParams['grid.alpha'] = 0
rcParams['xtick.bottom'] = True
rcParams['ytick.left'] = True

## Embed font
plt.rc('pdf', fonttype=42)

## Define new default settings
plt.rcParamsDefault = plt.rcParams

In [6]:
# Color maps
ch_YlRd=sb.cubehelix_palette(100, start=.7, rot=.25, gamma=0.6, hue=2, light=1, dark=0.05, as_cmap=True)
ch_Bl=sb.cubehelix_palette(100, start=2.65, rot=0, gamma=0.8, hue=1.8, light=1, dark=0, as_cmap=True)
ch_Bl2=sb.cubehelix_palette(100, start=2.75, rot=-.12, gamma=0.8, hue=1.8, light=1, dark=0, as_cmap=True)

# Setup R

In [7]:
#R
import os
os.environ['R_HOME'] = '/home/michi/Software/venvs/scAnalysis_sc1.9_ad0.8_mu0.1.2_md0.2_R4.1_FVF/lib/R' #path to your R installation

import rpy2
import rpy2.robjects as ro
import rpy2.rinterface_lib.callbacks
from rpy2.robjects import pandas2ri
import anndata2ri

## R settings

### Ignore R warning messages
#### Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

### Automatically convert rpy2 outputs to pandas dataframes
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

In [8]:
%%R

.libPaths()

[1] "/home/michi/Software/venvs/scAnalysis_sc1.9_ad0.8_mu0.1.2_md0.2_R4.1_FVF/lib/R/library"


In [9]:
%%R
library(scry)

# Parallelization
library("BiocParallel.FutureParam")
register(FutureParam())
plan(multicore, workers=8)
options(future.globals.maxSize = 2 * 1024 ^ 3) # for 50 Gb RAM

sessionInfo()

R version 4.1.1 (2021-08-10)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 20.04.5 LTS

Matrix products: default
BLAS/LAPACK: /home/michi/Software/venvs/scAnalysis_sc1.9_ad0.8_mu0.1.2_md0.2_R4.1_FVF/lib/libopenblasp-r0.3.18.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=de_DE.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=de_DE.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=de_DE.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=de_DE.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] tools     stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
[1] BiocParallel.FutureParam_0.2.1 BiocParallel_1.28.3           
[3] future_1.27.0                  scry_1.6.0                    

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.9                  parallelly_1.32.1          
 [3] BiocSingular_1

# Functions

In [10]:
def sparsify_all_layers(adata):
    """
    Loop trough all layers and make dense matrices sparse.
    """
          
    if not sci.sparse.issparse(adata.X):
        print('Sparsify .X...')
        adata.X = sci.sparse.csr_matrix(adata.X)
    else:
        print('.X already spase...')  
        
    for layer in list(adata.layers):
        if not sci.sparse.issparse(adata.layers[layer]):
            print('Sparsify ', layer)
            adata.layers[layer] = sci.sparse.csr_matrix(adata.layers[layer])
        else:
            print('Layer', layer, 'already spase...')

####################################################################################################################################################################################################################################
####################################################################################################################################################################################################################################
####################################################################################################################################################################################################################################
####################################################################################################################################################################################################################################

    
def print_r_session():
    ro.r('print(sessionInfo())')

# Load Data

In [11]:
adata=sc.read('/home/michi/Projects/scRNA-seq_iPSC_IGFRL-KO_Notebooks/Files/scRNA-seq_iPSC_IIR-KO_S6_adata_rmDoublets_normalized_integrated.h5ad')

In [13]:
adata = adata[~adata.obs.initial_cell_type.isin(['Non-Endocrine','Non-Epithelial'])].copy()

# Integration

In [14]:
genes = ['INS','GCG','TPH1','SST','ARX','NKX6-1','LMX1A','LMX1B','SLC18A1','ASCL1','GAP43','MKI67','KRT19','VIM']
marker_genes = ['INS','GCG','SST','ARX','TPH1','MKI67','CHGA','KRT19','VIM','EPCAM','GAP43','RPS26']

In [15]:
batch_key = 'batch'
labels_key = 'initial_cell_type'

## HVG Overlap

In [16]:
adata_hvg = adata.copy()

In [17]:
adata_raw_0 = ad.AnnData(X=adata[adata.obs.batch.isin(['0'])].layers['raw_counts'])
adata_raw_1 = ad.AnnData(X=adata[adata.obs.batch.isin(['1'])].layers['raw_counts'])

In [18]:
%%R -i adata_raw_0
sce = devianceFeatureSelection(adata_raw_0, assay='X')

/home/michi/Software/venvs/scAnalysis_sc1.9_ad0.8_mu0.1.2_md0.2_R4.1_FVF/lib/python3.8/site-packages/anndata2ri/r2py.py:102: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  return AnnData(exprs, obs, var, uns, obsm or None, layers=layers)
/home/michi/Software/venvs/scAnalysis_sc1.9_ad0.8_mu0.1.2_md0.2_R4.1_FVF/lib/python3.8/site-packages/anndata2ri/r2py.py:102: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  return AnnData(exprs, obs, var, uns, obsm or None, layers=layers)


In [19]:
binomial_deviance_0 = ro.r('rowData(sce)$binomial_deviance').T

In [20]:
idx = binomial_deviance_0.argsort()[-4000:]
mask = np.zeros(adata_hvg.var_names.shape, dtype=bool)
mask[idx] = True

adata_hvg.var['highly_deviant_0'] = mask

In [21]:
%%R -i adata_raw_1
sce = devianceFeatureSelection(adata_raw_1, assay='X')

/home/michi/Software/venvs/scAnalysis_sc1.9_ad0.8_mu0.1.2_md0.2_R4.1_FVF/lib/python3.8/site-packages/anndata2ri/r2py.py:102: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  return AnnData(exprs, obs, var, uns, obsm or None, layers=layers)
/home/michi/Software/venvs/scAnalysis_sc1.9_ad0.8_mu0.1.2_md0.2_R4.1_FVF/lib/python3.8/site-packages/anndata2ri/r2py.py:102: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  return AnnData(exprs, obs, var, uns, obsm or None, layers=layers)


In [22]:
binomial_deviance_1 = ro.r('rowData(sce)$binomial_deviance').T

In [23]:
idx = binomial_deviance_1.argsort()[-4000:]
mask = np.zeros(adata_hvg.var_names.shape, dtype=bool)
mask[idx] = True

adata_hvg.var['highly_deviant_1'] = mask

In [24]:
hvg_list = list(set(adata_hvg.var[adata_hvg.var['highly_deviant_0']].index).intersection(set(adata_hvg.var[adata_hvg.var['highly_deviant_1']].index)))

In [25]:
# Get HVGs and overlap with cell cycle & ambient genes

## HVGs
adata_hvg.var['highly_variable'] = adata_hvg.var_names.isin(hvg_list)
hvgs = pd.Series(adata_hvg.var_names[adata_hvg.var['highly_variable']])
print('\nHighly variable genes before filtering:',adata_hvg.var.loc[:,'highly_variable'].value_counts()[1])

# overlap HVGs with CC genes
hvcc = list(hvgs[hvgs.isin(all_cc_genes)])
print('\nHighly variable cell cycle genes:',len(hvcc),'\n',hvcc)

# overlap HVGs with ambient genes
hvambi = list(hvgs[hvgs.isin(list(adata_hvg[:,adata_hvg.var['is_ambient'] == True].var_names))])
print('\nHighly variable ambient genes:',len(hvambi),'\n',hvambi)

# remove cell cycle genes
adata_hvg.var.loc[hvcc,'highly_variable'] = False

# # remove ambient genes
# adata_hvg.var.loc[hvambi,'highly_variable'] = False

print('\nHighly variable genes after filtering:',adata_hvg.var.loc[:,'highly_variable'].value_counts()[1])


Highly variable genes before filtering: 3638

Highly variable cell cycle genes: 184 
 ['TOP1', 'VCL', 'MYCBP2', 'HMGB2', 'RHEB', 'TSC22D1', 'BRD7', 'TULP4', 'NT5DC1', 'CTSD', 'SEC62', 'CADM1', 'CFLAR', 'USP13', 'LNPEP', 'GNB1', 'TUBB', 'PHIP', 'KDM5B', 'NUMA1', 'NKTR', 'CEP57', 'CASP3', 'SSR3', 'BCLAF1', 'CBX5', 'CDC42', 'LARP7', 'GADD45A', 'MSL1', 'PTMS', 'EIF4E', 'TOP2A', 'DNAJB6', 'ZBTB7A', 'CCDC88A', 'TTLL7', 'DCAF16', 'CEP350', 'RAN', 'HMGCR', 'CENPF', 'CTR9', 'FXR1', 'TUBB4B', 'CTCF', 'DNAJA1', 'LARP1', 'TMPO', 'LMO4', 'POM121', 'RERE', 'TUBB2A', 'HIPK2', 'ZNF24', 'INSR', 'THRAP3', 'KLF6', 'NR3C1', 'NFIC', 'BUB3', 'REEP1', 'UBL3', 'AHI1', 'PPP6R3', 'KRAS', 'ZNF207', 'SFPQ', 'CALD1', 'ZBED5', 'EIF4EBP2', 'UBE2C', 'BTBD3', 'LMNA', 'NIPBL', 'AMD1', 'CCDC14', 'HMGB3', 'TXNRD1', 'ATF7IP', 'HP1BP3', 'DYNLL1', 'RSRC2', 'PSMD11', 'CAPN7', 'PPP2R2A', 'HSPA8', 'PPP1R2', 'ZNF593', 'YWHAH', 'PLIN3', 'RBM8A', 'ARL6IP1', 'SPTBN1', 'CYTH2', 'VPS25', 'PNN', 'KPNB1', 'SYNCRIP', 'TAF15', 'CERS6',

In [26]:
sc.pp.pca(adata_hvg, svd_solver='arpack', use_highly_variable=True)

computing PCA
    on highly variable genes
    with n_comps=50
    finished (0:00:07)


In [27]:
sc.pp.neighbors(adata_hvg)
sc.tl.leiden(adata_hvg)

computing neighbors
    using 'X_pca' with n_pcs = 50
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:01:26)
running Leiden clustering
    finished: found 14 clusters and added
    'leiden', the cluster labels (adata.obs, categorical) (0:00:00)


In [28]:
sc.tl.umap(adata_hvg)

computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:07)


In [ ]:
sc.pl.umap(adata_hvg, color=['sample','leiden'] + genes, size=10, add_outline=True, alpha=1, outline_width=(0.3, 0.0), ncols=4, color_map=ch_YlRd)

In [ ]:
for genotype in set(adata_hvg.obs['genotype']):
    palette = list(adata_hvg.uns['initial_cell_type_colors'])#['#1f77b4','#7ba2cd','#becfe6', '#279e68','#7abf98','#bddfca','#d62728','#ff7f0e', '#8c564b','#aa40fc','#cd85ff', '#17becf',  '#b5bd61', '#dcddaf', '#c6c6c6', '#919191',  '#5e5e5e']
    sc.pl.umap(adata_hvg[adata_hvg.obs['genotype']==genotype], color=['initial_cell_type'], title= genotype, size=12, add_outline=True, alpha=0.7, outline_width=(0.3, 0.0), ncols=4, show=True, palette=palette,
                    ax=sc.pl.umap(adata_hvg, color=['initial_cell_type'], size=5, legend_loc=None, add_outline=False, alpha=0.3, outline_width=(0.3, 0.0), ncols=4, show=False))

## SCVI Tools

In [ ]:
adata_scvi = adata.copy()

In [ ]:
scvi.model.SCVI.setup_anndata(adata_scvi, layer='raw_counts', batch_key=batch_key)

In [ ]:
model_scvi = scvi.model.SCVI(adata_scvi, n_hidden=512, n_layers=2, n_latent=30, gene_likelihood='nb', dispersion='gene-batch')
print(model_scvi)
model_scvi.view_anndata_setup()

In [ ]:
model_scvi.train()

In [ ]:
adata_scvi.obsm['X_scVI'] = model_scvi.get_latent_representation()

In [ ]:
sc.pp.neighbors(adata_scvi, use_rep='X_scVI')
sc.tl.leiden(adata_scvi)

In [ ]:
sc.tl.umap(adata_scvi)

In [ ]:
sc.pl.umap(adata_scvi, color=['sample','leiden'] + marker_genes, size=10, add_outline=True, alpha=1, outline_width=(0.3, 0.0), ncols=4)

In [ ]:
for genotype in set(adata_scvi.obs['genotype']):
    palette = list(adata_scvi.uns['initial_cell_type_colors'])#['#1f77b4','#7ba2cd','#becfe6', '#279e68','#7abf98','#bddfca','#d62728','#ff7f0e', '#8c564b','#aa40fc','#cd85ff', '#17becf',  '#b5bd61', '#dcddaf', '#c6c6c6', '#919191',  '#5e5e5e']
    sc.pl.umap(adata_scvi[adata_scvi.obs['genotype']==genotype], color=['initial_cell_type'], title= genotype, size=12, add_outline=True, alpha=0.7, outline_width=(0.3, 0.0), ncols=4, show=True, palette=palette,
                    ax=sc.pl.umap(adata_scvi, color=['initial_cell_type'], size=5, legend_loc=None, add_outline=False, alpha=0.3, outline_width=(0.3, 0.0), ncols=4, show=False))

## scANVI

In [ ]:
adata_scanvi = adata.copy()

In [ ]:
model_scanvi = scvi.model.SCANVI.from_scvi_model(
    model_scvi,
    adata=adata_scanvi, 
    labels_key=labels_key, 
    unlabeled_category='unlabelled'
)
print(model_scanvi)
model_scanvi.view_anndata_setup()

In [ ]:
model_scanvi.train(max_epochs=30)

In [ ]:
adata_scanvi.obsm['X_scANVI'] = model_scanvi.get_latent_representation()

In [ ]:
sc.pp.neighbors(adata_scanvi, use_rep='X_scANVI')
sc.tl.leiden(adata_scanvi)

In [ ]:
sc.tl.umap(adata_scanvi)

In [ ]:
sc.pl.umap(adata_scanvi, color=['sample','leiden'] + marker_genes, size=10, add_outline=True, alpha=1, outline_width=(0.3, 0.0), ncols=4)

In [ ]:
for genotype in set(adata_scanvi.obs['genotype']):
    palette = list(adata_scanvi.uns['initial_cell_type_colors'])#['#1f77b4','#7ba2cd','#becfe6', '#279e68','#7abf98','#bddfca','#d62728','#ff7f0e', '#8c564b','#aa40fc','#cd85ff', '#17becf',  '#b5bd61', '#dcddaf', '#c6c6c6', '#919191',  '#5e5e5e']
    sc.pl.umap(adata_scanvi[adata_scanvi.obs['genotype']==genotype], color=['initial_cell_type'], title= genotype, size=12, add_outline=True, alpha=0.7, outline_width=(0.3, 0.0), ncols=4, show=True, palette=palette,
                    ax=sc.pl.umap(adata_scanvi, color=['initial_cell_type'], size=5, legend_loc=None, add_outline=False, alpha=0.3, outline_width=(0.3, 0.0), ncols=4, show=False))

### Label Prediction

In [ ]:
adata_scanvi_label = adata.copy()

In [ ]:
sc.pl.umap(adata_scanvi, color=['sample','wt_cell_type'], size=10, add_outline=True, alpha=1, outline_width=(0.3, 0.0), ncols=4)

In [ ]:
model_scanvi = scvi.model.SCANVI.from_scvi_model(
    model_scvi,
    adata=adata_scanvi_label, 
    labels_key='wt_cell_type', 
    unlabeled_category='unlabelled'
)
print(model_scanvi)
model_scanvi.view_anndata_setup()

In [ ]:
model_scanvi.train(max_epochs=30)

In [ ]:
adata_scanvi_label.obsm['X_scANVI'] = model_scanvi.get_latent_representation()

In [ ]:
sc.pp.neighbors(adata_scanvi_label, use_rep='X_scANVI')
sc.tl.leiden(adata_scanvi_label)

In [ ]:
sc.tl.umap(adata_scanvi_label)

In [ ]:
sc.pl.umap(adata_scanvi_label, color=['sample','leiden'] + marker_genes, size=10, add_outline=True, alpha=1, outline_width=(0.3, 0.0), ncols=4)

In [ ]:
adata_scanvi_label.obs['predicted_cell_type'] = model_scanvi.predict()

In [ ]:
sc.pl.umap(adata_scanvi_label, color=['initial_cell_type','predicted_cell_type'], size=10, add_outline=True, alpha=1, outline_width=(0.3, 0.0), ncols=4, wspace=0.6)

In [ ]:
adata_scanvi_label.obs.initial_cell_type.value_counts()

In [ ]:
adata_scanvi_label.obs.predicted_cell_type.value_counts()

## Scanorama

In [ ]:
adata_scanorama = adata.copy()

In [ ]:
sce.pp.scanorama_integrate(adata_scanorama, key=batch_key,batch_size=50000)

In [ ]:
sc.pp.neighbors(adata_scanorama, use_rep='X_scanorama')
sc.tl.leiden(adata_scanorama)

In [ ]:
sc.tl.umap(adata_scanorama)

In [ ]:
sc.pl.umap(adata_scanorama, color=['sample','leiden'] + marker_genes, size=10, add_outline=True, alpha=1, outline_width=(0.3, 0.0), ncols=4)

## Harmony

In [ ]:
adata_harmony = adata.copy()

In [ ]:
sce.pp.harmony_integrate(adata_harmony, key=batch_key, adjusted_basis='X_harmony')

In [ ]:
sc.pp.neighbors(adata_harmony, use_rep='X_harmony')
sc.tl.leiden(adata_harmony)

In [ ]:
sc.tl.umap(adata_harmony)

In [ ]:
sc.pl.umap(adata_harmony, color=['sample','leiden'] + marker_genes, size=10, add_outline=True, alpha=1, outline_width=(0.3, 0.0), ncols=4)

# Benchmark

In [ ]:
import scib
sc.set_figure_params(scanpy=True, frameon=False, vector_friendly=False ,transparent=True, dpi=150, dpi_save=300)

In [ ]:
metrics_scvi = scib.metrics.metrics_fast(
    adata, adata_scvi, batch_key, labels_key, embed='X_scVI'
)
metrics_scanvi = scib.metrics.metrics_fast(
    adata, adata_scanvi, batch_key, labels_key, embed='X_scANVI'
)
metrics_scanorama = scib.metrics.metrics_fast(
    adata, adata_scanorama, batch_key, labels_key, embed='X_scanorama'
)
metrics_harmony = scib.metrics.metrics_fast(
    adata, adata_harmony, batch_key, labels_key, embed='X_harmony'
)
metrics_hvg = scib.metrics.metrics_fast(adata, adata_hvg, batch_key, labels_key)
metrics_unintegrated = scib.metrics.metrics_fast(adata, adata, batch_key, labels_key)

In [ ]:
# Concatenate metrics results
metrics = pd.concat(
    [metrics_scvi, metrics_scanvi, metrics_scanorama, metrics_harmony, metrics_hvg, metrics_unintegrated],
    axis='columns',
)
# Set methods as column names
metrics = metrics.set_axis(
    ['scVI', 'scANVI', 'Scanorama', 'Harmony', 'HVG Overlap', 'Unintegrated'], axis='columns'
)
# Select only the fast metrics
metrics = metrics.loc[
    [
        'ASW_label',
        'ASW_label/batch',
        'PCR_batch',
        'isolated_label_silhouette',
        'graph_conn',
        "hvg_overlap",
    ],
    :,
]
# Transpose so that metrics are columns and methods are rows
metrics = metrics.T
metrics

In [ ]:
metrics_scaled = (metrics - metrics.min()) / (metrics.max() - metrics.min())
metrics_scaled.style.background_gradient(cmap='Blues')

In [ ]:
metrics.style.background_gradient(cmap='Blues')

In [ ]:
metrics['Batch'] = metrics[
    ['ASW_label/batch', 'PCR_batch', 'graph_conn']
].mean(axis=1)
metrics['Bio'] = metrics[
    ['ASW_label', 'isolated_label_silhouette', "hvg_overlap",]
].mean(axis=1)
metrics.style.background_gradient(cmap='Blues')

In [ ]:
fig, ax = plt.subplots()
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
metrics.plot.scatter(
    x='Batch',
    y='Bio',
    c=range(len(metrics)),
    colormap=mpl.cm.get_cmap('Set1'),
    ax=ax,
)
anno = []
for k, v in metrics[['Batch', 'Bio']].iterrows():
    anno.append(plt.text(
        v['Batch'],
        v['Bio'],
        k,
        fontsize=10,
    ))
adjust_text(anno, expand_points=(3,3), expand_text=(3,3), expand_objects=(3,3), precision=0.00001, lim=5000, autoalign='xy', arrowprops=dict(arrowstyle="-",  color='k',  lw=0.5), ax=ax)

In [ ]:
metrics["Overall"] = 0.4 * metrics["Batch"] + 0.6 * metrics["Bio"]
metrics.style.background_gradient(cmap="Blues")

In [ ]:
metrics.plot.bar(y='Overall')

In [ ]:
metrics_scaled['Batch'] = metrics_scaled[
    ['ASW_label/batch', 'PCR_batch', 'graph_conn']
].mean(axis=1)
metrics_scaled['Bio'] = metrics_scaled[
    ['ASW_label', 'isolated_label_silhouette', "hvg_overlap",]
].mean(axis=1)
metrics_scaled.style.background_gradient(cmap='Blues')

In [ ]:
fig, ax = plt.subplots()
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
metrics_scaled.plot.scatter(
    x='Batch',
    y='Bio',
    c=range(len(metrics_scaled)),
    colormap=mpl.cm.get_cmap('Set1'),
    ax=ax,
)

for k, v in metrics_scaled[['Batch', 'Bio']].iterrows():
    ax.annotate(
        k,
        v,
        xytext=(6, -3),
        textcoords='offset points',
        family='sans-serif',
        fontsize=12,
    )

In [ ]:
metrics_scaled["Overall"] = 0.4 * metrics_scaled["Batch"] + 0.6 * metrics_scaled["Bio"]
metrics_scaled.style.background_gradient(cmap="Blues")

In [ ]:
metrics_scaled.plot.bar(y='Overall')

# Collect Results

In [83]:
# get latent representaions
adata.obsm['X_scVI'] = model_scvi.get_latent_representation()
adata.obsm['X_scANVI'] = model_scanvi.get_latent_representation()
adata.obsm['X_scanorama'] = adata_scanorama.obsm['X_scanorama']
adata.obsm['X_harmony'] = adata_harmony.obsm['X_harmony']

In [84]:
# get denoised expression from scvi
adata.layers['scvi_counts'] = model_scvi.get_normalized_expression(library_size=10e4, n_samples=10)

In [85]:
# get label predictions for KO
adata.obs['predicted_cell_type'] = adata_scanvi_label.obs['predicted_cell_type']

# Save

In [86]:
sparsify_all_layers(adata)

.X already spase...
Layer ambiguous already spase...
Layer log_raw_counts already spase...
Layer matrix already spase...
Layer raw_counts already spase...
Layer scran_counts already spase...
Layer sct_counts already spase...
Layer sct_logcounts already spase...
Layer sct_scale_data already spase...
Sparsify  scvi_counts
Layer spliced already spase...
Layer unspliced already spase...


In [87]:
# Save
sc.write('/home/michi/Projects/scRNA-seq_iPSC_IGFRL-KO_Notebooks/Files/scRNA-seq_iPSC_IIR-KO_S6_adata_rmDoublets_normalized_integrated_endocrine', adata)

# Session Info

In [88]:
session_info.show()

/home/michi/Software/venvs/scAnalysis_sc1.9_ad0.8_mu0.1.2_md0.2_R4.1_FVF/lib/python3.8/site-packages/jsonschema/__init__.py:43: DeprecationWarning: Accessing jsonschema.__version__ is deprecated and will be removed in a future release. Use importlib.metadata directly to query for jsonschema's version.
  warnings.warn(


In [89]:
print_r_session()

R version 4.1.1 (2021-08-10)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 20.04.5 LTS

Matrix products: default
BLAS/LAPACK: /home/michi/Software/venvs/scAnalysis_sc1.9_ad0.8_mu0.1.2_md0.2_R4.1_FVF/lib/libopenblasp-r0.3.18.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=de_DE.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=de_DE.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=de_DE.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=de_DE.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats4    tools     stats     graphics  grDevices utils     datasets 
[8] methods   base     

other attached packages:
 [1] Matrix_1.4-1                   SingleCellExperiment_1.16.0   
 [3] SummarizedExperiment_1.24.0    Biobase_2.54.0                
 [5] GenomicRanges_1.46.1           GenomeInfoDb_1.30.1           
 [7] IRanges_2.28.0                 MatrixGen